## Image Classification with Convolutional Neural Networks
Now we will do image classification on the mnist data that we have been looking at in the lectures.

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras import backend as K

We are trying to work out what number an image is. There are 10 possible classes so rather than having binary output labels, we will need vectors of size 10. You have seen the other parameters before.

In [ ]:
num_classes = 10
batch_size = 128
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

We can load the mnist dataset directly from Keras.

In [ ]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train.size

We have 60000 images to train on, and 60000 more to test on. For now, we will work on a small subset of these.

In [ ]:
x_train = x_train[:5000, :,:]
y_train = y_train[:5000]
x_test = x_test[:5000, :,:]
y_test = y_test[:5000]
x_train.shape

Convolution neural networks are usually done on colour image data, this is usually `height x width x 3` where we have the three colour channels: red, green and blue. Keras convolutional layers are built for this, so we need to add an extra channel dimension.

In [ ]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 # This converts the pixel values from between 0 and 255 to between 0 and 1
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

We want to set up y_train and y_test to have a vector of size 10 for each input sample.

y-train is the training label. For an image of a handwritten 0, it should be `[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]`; for a 1, it should be `[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]` etc.

y_test is the output vector of confidence scores that the model will return. For an image of a one, we might expect something like `[0, 0.9, 0, 0, 0, 0, 0, 0.1, 0, 0]`. The model is 90% sure it is a 1, and 10% sure it is a 7 (because some people write a 7 similarly to a 1)

In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
y_train.shape

### Building the model
Again we need a sequential model. Now however, instead of `Dense` layers, we will use `Conv2d` layers. These are very similar to the Dense layers we saw previously, but have an added variable `kernel_size`. Everything else in these layers you have seen before. Note that because we switch from binary classification to multi-class classification, the final activation function becomes a softmax.

Recall with convolutional layers that we take a feature and scan across the image, looking for parts of the image that look similar to the feature. The mathematical term for this feature is a `kernel`. A kernel size of `(3,3)` will give a 3x3 matrix on our data. So a 3x3 matrix will scan across the whole image.

Note that when we have colour images, a kernel size of `(3, 3)` is a 3x3x3 tensor, as we have 3 colour channels. You don't need to worry about this, keras will automatically match up the kernel size with the number of channels.
#### Max Pooling
We also have a Max Pooling layer, to help reduce the image size. Max pooling, with a pool size of `(2,2)`, splits the image up into 2x2 chunks, finds the maximum number of those chunks, and deletes the other values. This will have the dimensions of the images you are working with
You can also do average pooling, but it is a little less efficient.



#### Visualising the model
Use model.summary() to see all the input dimensions at each layer

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.3))
model.add(Flatten()) # This line is to convert from matrices to vectors
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax')) # we are working with vectors now, so we use a Dense layer instead of Conv2d
model.summary()

In [ ]:
model.compile(loss=categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

### Training the model
We fit the model in the same way as in the previous notebook (`verbose = 1` hasn't got anything to do with fitting the model, just how results are presented). 

The biggest difference will be how much slower the network is, even with our small subset of the data. Try switching to a GPU via Edit ->  Notebook Settings. You can see this visually in my 'Putting it all together' video.

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

### Summary
Even though convolutional neural networks are more complicated conceptually than regular neural networks, they are not too difficult to code.

For all of these tasks, the most laborious part has been setting up the data in such a way that it can be processed. Once that has been managed, Keras will do all the work.

### Exercise 1
As before, do what you can to increase the accuracy of the model. Play around with more layers, more channels, a different batch size. Try to get as close to 100 as you can. Then try using more data, depending on how powerful the machine you have is. 

### Exercise 2
Convert the numpy data to 784x1 vectors as we did in the lecture. Compose a neural network (like we did in the previous notebook) and try to get as close to you test accuracy with convolutional neural networks as you can.